In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=398aa80ca464fb80bbd95043e637e792e49ef3decc0c6afbfd55fa72b706bbba
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ca_train_data_final.csv to ca_train_data_final.csv


In [ ]:
spark=SparkSession.builder.appName("CustomerDataOperation").getOrCreate()
df = spark.read.csv("/content/ca_train_data_final.csv",header=True, inferSchema=True)
df.show(5)

+------------+------------+---------+-------------+---+---------+------+--------+------------+-----------+--------------+---------+-------------+------------+----------------+------+-------------+--------------------+-----------+---------+----------------+---------------------+------------------+-------------+---------------------------+--------------------------+-----------------------+------------------+----------------------+-------------------+---------------+-------------------------+-------------------------+-------------------+-----------------------+----------------+----------------+--------------+--------------+------------------+-----------------+-----------------+---------------+---------------+-------------------+---------+-----------------+
|CUSTOMER_NBR|CUST_SEGMENT|TREATMENT|net_sales_cst|qty|net_sales|margin|Response|RECENCY_DAYS|RECENCY_MTH|YEARS_SINCE_FP|REACT_MTH|CUST_CATEGORY|FL_DONT_CALL|FL_TP_CARDHOLDER|FL_EMP|FL_HOUSE_CARD|FL_ACTIVE_HOUSE_CARD|CD_RELIGION|CD_ROLL

In [ ]:
df.printSchema()
df.show(5)

root
 |-- CUSTOMER_NBR: integer (nullable = true)
 |-- CUST_SEGMENT: string (nullable = true)
 |-- TREATMENT: integer (nullable = true)
 |-- net_sales_cst: double (nullable = true)
 |-- qty: double (nullable = true)
 |-- net_sales: double (nullable = true)
 |-- margin: double (nullable = true)
 |-- Response: double (nullable = true)
 |-- RECENCY_DAYS: double (nullable = true)
 |-- RECENCY_MTH: double (nullable = true)
 |-- YEARS_SINCE_FP: double (nullable = true)
 |-- REACT_MTH: double (nullable = true)
 |-- CUST_CATEGORY: string (nullable = true)
 |-- FL_DONT_CALL: double (nullable = true)
 |-- FL_TP_CARDHOLDER: double (nullable = true)
 |-- FL_EMP: double (nullable = true)
 |-- FL_HOUSE_CARD: double (nullable = true)
 |-- FL_ACTIVE_HOUSE_CARD: double (nullable = true)
 |-- CD_RELIGION: string (nullable = true)
 |-- CD_ROLLUP: string (nullable = true)
 |-- CD_INCOME_EST_HH: double (nullable = true)
 |-- CD_1ST_INDIVIDUAL_AGE: double (nullable = true)
 |-- FL_CC_GOLD_PREMIUM: double (n

In [ ]:
df.select("CUSTOMER_NBR","RECENCY_DAYS","net_sales","margin").show(5)

+------------+------------+---------+------+
|CUSTOMER_NBR|RECENCY_DAYS|net_sales|margin|
+------------+------------+---------+------+
|     8593997|       165.0|      0.0|   0.0|
|    31823408|       225.0|      0.0|   0.0|
|   405520555|        76.0|      0.0|   0.0|
|    16149066|         2.0|      0.0|   0.0|
|   198912146|        93.0|      0.0|   0.0|
+------------+------------+---------+------+
only showing top 5 rows



In [ ]:
df=df.withColumnRenamed("CUSTOMER_NBR","Customer_Number")
df

DataFrame[Customer_Number: int, CUST_SEGMENT: string, TREATMENT: int, net_sales_cst: double, qty: double, net_sales: double, margin: double, Response: double, RECENCY_DAYS: double, RECENCY_MTH: double, YEARS_SINCE_FP: double, REACT_MTH: double, CUST_CATEGORY: string, FL_DONT_CALL: double, FL_TP_CARDHOLDER: double, FL_EMP: double, FL_HOUSE_CARD: double, FL_ACTIVE_HOUSE_CARD: double, CD_RELIGION: string, CD_ROLLUP: string, CD_INCOME_EST_HH: double, CD_1ST_INDIVIDUAL_AGE: double, FL_CC_GOLD_PREMIUM: double, FL_CC_REGULAR: double, CD_IP_INDIVIDUAL_OCCUPATION: string, CD_IP_INDIVIDUAL_EDUCATION: double, CD_MARITAL_STATUS_IN_HH: string, CD_NUM_OF_CHILDREN: double, NU_LENGTH_OF_RESIDENCE: double, FL_WOMAN_PLUS_SIZES: double, FL_WPS_BUSINESS: double, FL_MOB_APPAREL_PLUS_WOMEN: double, FL_MOB_GENERAL_GIFTS_MDSE: double, CD_MAIL_ORDER_BUYER: double, CD_MAIL_ORDER_RESPONDER: double, spring_SALES_pct: double, summer_SALES_pct: double, fall_SALES_pct: double, xmas_SALES_pct: double, postxmas_SALES_

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType

schema = StructType([
    StructField("Customer_Number", StringType(), True),
    StructField("RECENCY_DAYS", IntegerType(), True),
    StructField("RECENCY_MTH", IntegerType(), True),
    StructField("YEARS_SINCE_FP", IntegerType(), True),
    StructField("net_sales", FloatType(), True),
    StructField("margin", FloatType(), True),
    StructField("Response", IntegerType(), True),
    StructField("CUST_SEGMENT", IntegerType(), True),
])

df = spark.read.csv("/content/ca_train_data_final.csv", schema=schema, header=True)
df.printSchema()


root
 |-- Customer_Number: string (nullable = true)
 |-- RECENCY_DAYS: integer (nullable = true)
 |-- RECENCY_MTH: integer (nullable = true)
 |-- YEARS_SINCE_FP: integer (nullable = true)
 |-- net_sales: float (nullable = true)
 |-- margin: float (nullable = true)
 |-- Response: integer (nullable = true)
 |-- CUST_SEGMENT: integer (nullable = true)



In [ ]:
from pyspark.sql.functions import min, max, col
df.filter(col("RECENCY_DAYS").isNull()).show()
df_filtered = df.filter(col("RECENCY_DAYS").isNotNull())
df_subset = df_filtered.select("Customer_Number", "RECENCY_DAYS", "YEARS_SINCE_FP")

# Calculate the min and max values for RECENCY_DAYS
min_val = df_filtered.agg(min("RECENCY_DAYS")).collect()[0][0]
max_val = df_filtered.agg(max("RECENCY_DAYS")).collect()[0][0]

print("Min RECENCY_DAYS:", min_val)
print("Max RECENCY_DAYS:", max_val)

if min_val is not None and max_val is not None:
    # Normalize RECENCY_DAYS
    df_normalized = df_subset.withColumn("normalized_RECENCY_DAYS",
        (col("RECENCY_DAYS") - min_val) / (max_val - min_val))
    df_normalized.show()
else:
    print("Min or Max value for RECENCY_DAYS could not be determined.")


+---------------+------------+--------------+---------+------+--------+
|Customer_Number|RECENCY_DAYS|YEARS_SINCE_FP|net_sales|margin|Response|
+---------------+------------+--------------+---------+------+--------+
|        8593997|        NULL|             1|      0.0|   0.0|    NULL|
|       31823408|        NULL|             1|      0.0|   0.0|    NULL|
|      405520555|        NULL|             1|      0.0|   0.0|    NULL|
|       16149066|        NULL|             1|      0.0|   0.0|    NULL|
|      198912146|        NULL|             1|      0.0|   0.0|    NULL|
|       31198436|        NULL|             1|      0.0|   0.0|    NULL|
|      204990635|        NULL|             1|      0.0|   0.0|    NULL|
|      187585915|        NULL|             1|    29.78|   2.0|    NULL|
|       32782196|        NULL|             1|      0.0|   0.0|    NULL|
|      143992588|        NULL|             1|      0.0|   0.0|    NULL|
|      407349528|        NULL|             1|   145.76|  13.0|  

In [ ]:
columns_to_rename = ["net_sales", "margin", "Response"]

for col_name in columns_to_rename:
    df = df.withColumnRenamed(col_name, f"{col_name}_2023")

df.show()


+---------------+------------+--------------+--------------+-----------+-------------+
|Customer_Number|RECENCY_DAYS|YEARS_SINCE_FP|net_sales_2023|margin_2023|Response_2023|
+---------------+------------+--------------+--------------+-----------+-------------+
|        8593997|        NULL|             1|           0.0|        0.0|         NULL|
|       31823408|        NULL|             1|           0.0|        0.0|         NULL|
|      405520555|        NULL|             1|           0.0|        0.0|         NULL|
|       16149066|        NULL|             1|           0.0|        0.0|         NULL|
|      198912146|        NULL|             1|           0.0|        0.0|         NULL|
|       31198436|        NULL|             1|           0.0|        0.0|         NULL|
|      204990635|        NULL|             1|           0.0|        0.0|         NULL|
|      187585915|        NULL|             1|         29.78|        2.0|         NULL|
|       32782196|        NULL|             

In [ ]:
df_filtered = df.filter((col("RECENCY_DAYS") > 200) |
                        ((col("CUST_SEGMENT") == "REACTIVATED") & (col("Response") != 0)))

df_filtered.show()


+---------------+------------+--------------+---------+------+--------+------------+
|Customer_Number|RECENCY_DAYS|YEARS_SINCE_FP|net_sales|margin|Response|CUST_SEGMENT|
+---------------+------------+--------------+---------+------+--------+------------+
+---------------+------------+--------------+---------+------+--------+------------+



In [ ]:
df_sorted = df.orderBy(col("CUST_SEGMENT").asc(), col("net_sales").desc())
df_sorted.show(10)


+---------------+------------+--------------+---------+------+--------+------------+
|Customer_Number|RECENCY_DAYS|YEARS_SINCE_FP|net_sales|margin|Response|CUST_SEGMENT|
+---------------+------------+--------------+---------+------+--------+------------+
|      197916555|        NULL|             1|  4926.85| 505.0|    NULL|        NULL|
|      197916555|        NULL|             1|  4926.85| 505.0|    NULL|        NULL|
|       54951231|        NULL|             1|   1516.2| 199.0|    NULL|        NULL|
|       57136053|        NULL|             1|  1450.63| 132.0|    NULL|        NULL|
|      408501017|        NULL|             1|  1267.71| 130.0|    NULL|        NULL|
|      405525934|        NULL|             1|  1032.78| 116.0|    NULL|        NULL|
|      408637654|        NULL|             1|   864.14|  93.0|    NULL|        NULL|
|      812556869|        NULL|             1|   836.58|  56.0|    NULL|        NULL|
|      183151591|        NULL|             1|   796.95|  75.0|   

In [ ]:
from pyspark.sql.functions import expr

median_margin = df.stat.approxQuantile("margin", [0.5], 0.01)[0]

df_filled = df.withColumn("margin", expr(f"IFNULL(margin, {median_margin})"))
df_filled.show()


+---------------+------------+--------------+---------+------+--------+------------+
|Customer_Number|RECENCY_DAYS|YEARS_SINCE_FP|net_sales|margin|Response|CUST_SEGMENT|
+---------------+------------+--------------+---------+------+--------+------------+
|        8593997|        NULL|             1|      0.0|   0.0|    NULL|        NULL|
|       31823408|        NULL|             1|      0.0|   0.0|    NULL|        NULL|
|      405520555|        NULL|             1|      0.0|   0.0|    NULL|        NULL|
|       16149066|        NULL|             1|      0.0|   0.0|    NULL|        NULL|
|      198912146|        NULL|             1|      0.0|   0.0|    NULL|        NULL|
|       31198436|        NULL|             1|      0.0|   0.0|    NULL|        NULL|
|      204990635|        NULL|             1|      0.0|   0.0|    NULL|        NULL|
|      187585915|        NULL|             1|    29.78|   2.0|    NULL|        NULL|
|       32782196|        NULL|             1|      0.0|   0.0|   

In [ ]:
df_grouped = df.filter(col("RECENCY_DAYS") > 50) \
    .groupBy("CUST_SEGMENT") \
    .agg({"net_sales": "avg", "Response": "sum"})
df_grouped.show()


+------------+-------------+--------------+
|CUST_SEGMENT|sum(Response)|avg(net_sales)|
+------------+-------------+--------------+
+------------+-------------+--------------+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
schema = StructType([
    StructField("Customer_Number", StringType(), True),
    StructField("LOYALTY_SCORE", IntegerType(), True)
])
data = [("12345", 60), ("54321", 40), ("98765", 55), ("67890", 80)]
df2 = spark.createDataFrame(data, schema)
df2.show()


+---------------+-------------+
|Customer_Number|LOYALTY_SCORE|
+---------------+-------------+
|          12345|           60|
|          54321|           40|
|          98765|           55|
|          67890|           80|
+---------------+-------------+



In [ ]:
from pyspark.sql.functions import col
df_joined = df.join(df2, "Customer_Number", "left")
df_filtered = df_joined.filter(col("LOYALTY_SCORE") > 50)
df_filtered.show()


+---------------+------------+--------------+---------+------+--------+------------+-------------+
|Customer_Number|RECENCY_DAYS|YEARS_SINCE_FP|net_sales|margin|Response|CUST_SEGMENT|LOYALTY_SCORE|
+---------------+------------+--------------+---------+------+--------+------------+-------------+
|          67890|        NULL|             1|      0.0|   0.0|    NULL|        NULL|           80|
+---------------+------------+--------------+---------+------+--------+------------+-------------+



In [ ]:
df_updated = df.withColumn("Response", expr("CASE WHEN RECENCY_DAYS < 50 THEN 1 ELSE Response END")) \
               .withColumn("net_sales", expr("CASE WHEN RECENCY_DAYS < 50 THEN net_sales * 1.1 ELSE net_sales END"))
df_updated.show()


+---------------+------------+--------------+------------------+------+--------+------------+
|Customer_Number|RECENCY_DAYS|YEARS_SINCE_FP|         net_sales|margin|Response|CUST_SEGMENT|
+---------------+------------+--------------+------------------+------+--------+------------+
|        8593997|        NULL|             1|               0.0|   0.0|    NULL|        NULL|
|       31823408|        NULL|             1|               0.0|   0.0|    NULL|        NULL|
|      405520555|        NULL|             1|               0.0|   0.0|    NULL|        NULL|
|       16149066|        NULL|             1|               0.0|   0.0|    NULL|        NULL|
|      198912146|        NULL|             1|               0.0|   0.0|    NULL|        NULL|
|       31198436|        NULL|             1|               0.0|   0.0|    NULL|        NULL|
|      204990635|        NULL|             1|               0.0|   0.0|    NULL|        NULL|
|      187585915|        NULL|             1|29.780000686645

In [ ]:
df_clv = df.withColumn("Customer_Lifetime_Value",
                       expr("CASE WHEN CUST_SEGMENT = 'REACTIVATED' THEN net_sales * margin ELSE net_sales / 2 END"))
df_clv.show()


+---------------+------------+--------------+---------+------+--------+------------+-----------------------+
|Customer_Number|RECENCY_DAYS|YEARS_SINCE_FP|net_sales|margin|Response|CUST_SEGMENT|Customer_Lifetime_Value|
+---------------+------------+--------------+---------+------+--------+------------+-----------------------+
|        8593997|        NULL|             1|      0.0|   0.0|    NULL|        NULL|                    0.0|
|       31823408|        NULL|             1|      0.0|   0.0|    NULL|        NULL|                    0.0|
|      405520555|        NULL|             1|      0.0|   0.0|    NULL|        NULL|                    0.0|
|       16149066|        NULL|             1|      0.0|   0.0|    NULL|        NULL|                    0.0|
|      198912146|        NULL|             1|      0.0|   0.0|    NULL|        NULL|                    0.0|
|       31198436|        NULL|             1|      0.0|   0.0|    NULL|        NULL|                    0.0|
|      204990635|  

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
window_spec = Window.partitionBy("CUST_SEGMENT").orderBy(col("net_sales").desc())
df_ranked = df.withColumn("rank", rank().over(window_spec))
df_ranked.show()


+---------------+------------+--------------+---------+------+--------+------------+----+
|Customer_Number|RECENCY_DAYS|YEARS_SINCE_FP|net_sales|margin|Response|CUST_SEGMENT|rank|
+---------------+------------+--------------+---------+------+--------+------------+----+
|      197916555|        NULL|             1|  4926.85| 505.0|    NULL|        NULL|   1|
|      197916555|        NULL|             1|  4926.85| 505.0|    NULL|        NULL|   1|
|       54951231|        NULL|             1|   1516.2| 199.0|    NULL|        NULL|   3|
|       57136053|        NULL|             1|  1450.63| 132.0|    NULL|        NULL|   4|
|      408501017|        NULL|             1|  1267.71| 130.0|    NULL|        NULL|   5|
|      405525934|        NULL|             1|  1032.78| 116.0|    NULL|        NULL|   6|
|      408637654|        NULL|             1|   864.14|  93.0|    NULL|        NULL|   7|
|      812556869|        NULL|             1|   836.58|  56.0|    NULL|        NULL|   8|
|      183

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg
window_spec = Window.partitionBy("Customer_Number").orderBy("RECENCY_MTH").rowsBetween(-2, 0)
df_rolling_avg = df.withColumn("rolling_avg_net_sales", avg("net_sales").over(window_spec))
df_rolling_avg.show()


+---------------+------------+-----------+--------------+---------+------+--------+------------+---------------------+
|Customer_Number|RECENCY_DAYS|RECENCY_MTH|YEARS_SINCE_FP|net_sales|margin|Response|CUST_SEGMENT|rolling_avg_net_sales|
+---------------+------------+-----------+--------------+---------+------+--------+------------+---------------------+
|       10000100|        NULL|          1|          NULL|      1.0| 16.18|    NULL|        NULL|                  1.0|
|      100010661|        NULL|          1|          NULL|      0.0|   0.0|    NULL|        NULL|                  0.0|
|       10001195|        NULL|          1|          NULL|      0.0|   0.0|    NULL|        NULL|                  0.0|
|       10001447|        NULL|          1|          NULL|      0.0|   0.0|    NULL|        NULL|                  0.0|
|      100021669|        NULL|          1|          NULL|      0.0|   0.0|    NULL|        NULL|                  0.0|
|       10002558|        NULL|          1|      